In [19]:
import random
import textwrap
import pandas as pd 
from ipywidgets import widgets, HBox
from pathlib import Path


In [20]:
df = pd.read_json('../../tmp/chief-aces/data/generated/evaluation/results.jsonl', lines=True)
print(len(df))
df.head()

100


,id,question,context,docs,answers,question_decomposition,predicted_reasoning,predicted_answer,score,n_hops,exact_match,f1
0,history_1859-902f2cd7-1fd4-4519-9611-0815f380cc95,How many percent are not female householder w...,"# \nThere were 88,296 households out of which ...","[{'idx': 0, 'text': '# There were 88,296 hous...","[90.9, 90.9, 9.1, 90.9]",[],The context states that 9.1% of households had...,90.9%,0,0,0,0.400000
1,history_2196-5d38562b-d4f0-43f9-b33f-387b56a226cc,Which group for households is larger: married ...,"# \nAs of 2000, there were 19,306 households o...","[{'idx': 0, 'text': '# As of 2000, there were...","[married couples living together, married coup...",[],The context provides percentages for different...,married couples living together,0,0,0,0.666667
2,history_3400-b46de4d7-0e58-4dac-b1c0-2deaba0defa8,What age categories were included in the senio...,"# \n, the gender distribution of the populatio...","[{'idx': 0, 'text': '# , the gender distribut...","[60 and 69 years old, 70 to 79, 80 to 89, 90 t...",[],The senior population distribution refers to t...,"60 to 69 years old, 70 to 79 years old, 80 to ...",0,0,0,0.666667
3,nfl_1672-666a4d2c-75af-49ab-b254-9cf6b3eb0f70,how many yards did collins complete?,# \n The Titans began their season at Heinz Fi...,"[{'idx': 0, 'text': '# The Titans began thei...","[16, 16]",[],The context mentions that Kerry Collins comple...,16,0,0,0,0.666667
4,history_1954-2c054bf0-1177-4f44-865c-a6e02d20494e,"How many more people, in terms of percentage, ...","# \nAs of the census of 2000, there were 19,23...","[{'idx': 0, 'text': '# As of the census of 20...","[95.09, 95.09]",[],The two biggest racial groups are White (82.39...,95.09%,0,0,0,0.666667


In [21]:
success_mask = df['f1'] > 0.5
fail_df = df[~success_mask]
success_df = df[success_mask]
print("Fail:", len(fail_df))
print("Success:", len(success_df))
print("Total:", len(df))
print("Mean F1:", f"{df['f1'].mean():.3f}")

Fail: 39
Success: 61
Total: 100
Mean F1: 0.538


In [22]:
def fixedwidth(text):
    return "\n".join(textwrap.wrap(text, width=80, replace_whitespace=False))

def format_row(row):
    output = []
    output.append(f"{row['id']} - {row['n_hops']} hops")
    output.append("="*80)
    output.append(fixedwidth(row['context']))
    output.append("="*80)
    output.append("Q: " + row['question'])
    output.append("Reference Answers: " + str(row['answers']))
    output.append("Predicted Answer: " + row['predicted_answer'])
    output.append("")
    output.append("# Scores")
    output.append("EM: {:.3f}".format(row['exact_match']))
    output.append("F1: {:.3f}".format(row['f1']))
    if 'analysis' in row:
        output.append("")
        output.append("# Analysis")
        output.append(fixedwidth(row['analysis']))
    return "\n".join(output)

def present_row(row):
    print(format_row(row))


def create_browse_app(df):
    def browse_failed(i=0):
        row = df.iloc[i]
        present_row(row)

    index = widgets.IntText(value=0, description='Index:')
    left_button = widgets.Button(description='Previous')
    right_button = widgets.Button(description='Next')

    def on_left_button_clicked(b):
        if index.value > 0:
            index.value -= 1

    def on_right_button_clicked(b):
        if index.value < len(df) - 1:
            index.value += 1

    left_button.on_click(on_left_button_clicked)
    right_button.on_click(on_right_button_clicked)

    ui = HBox([left_button, index, right_button])
    out = widgets.interactive_output(browse_failed, {'i': index})

    display(ui, out)

# Usage

inspect_df = fail_df.sort_values('f1')
create_browse_app(inspect_df)


Output()

In [24]:
inspect_df.iloc[0]

id                        history_2765-0e223311-f4ff-473f-8443-29d65d1a07f9
question                  How many percent of people were not African Am...
context                   # \nAs of 2011 the 88,112 residents of Orem ha...
docs                      [{'idx': 0, 'text': '# 
As of 2011 the 88,112 ...
answers                                                      [99.67, 99.67]
question_decomposition                                                   []
predicted_reasoning       The context provides the racial composition of...
predicted_answer                                                      99.1%
score                                                                     0
n_hops                                                                    0
exact_match                                                               0
f1                                                                      0.0
Name: 43, dtype: object